In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import icomo
import jax.numpy as jnp

import sys
import os
from matplotlib.ticker import FuncFormatter
from mpl_toolkits.axes_grid1.inset_locator import inset_axes


sys.path.append("../")
from tapm import model_HIVandSTI
from tapm import sti_infection_results_H
from tapm import utils

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': 
INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': UNIMPLEMENTED: LoadPjrtPlugin is not implemented on windows yet.


In [2]:
args, y0 = utils.read_params(filename="../src/tapm/HIVandSTI/model_HIVandSTI.py")
integrator = model_HIVandSTI.setup_model(args, y0)

INFO:tapm.HIVandSTI.model_HIVandSTI:Model setup complete and ready for simulation


In [4]:
output = integrator(y0=y0, constant_args=args)

L and ll in function M 0 0
L and ll in function M 0 1
L and ll in function M 0 2
L and ll in function M 0 3
L and ll in function M 1 0
L and ll in function M 1 1
L and ll in function M 1 2
L and ll in function M 1 3
L and ll in function M 2 0
L and ll in function M 2 1
L and ll in function M 2 2
L and ll in function M 2 3
L and ll in function M 3 0
L and ll in function M 3 1
L and ll in function M 3 2
L and ll in function M 3 3
L and ll in function M 0 0
L and ll in function M 0 1
L and ll in function M 0 2
L and ll in function M 0 3
L and ll in function M 1 0
L and ll in function M 1 1
L and ll in function M 1 2
L and ll in function M 1 3
L and ll in function M 2 0
L and ll in function M 2 1
L and ll in function M 2 2
L and ll in function M 2 3
L and ll in function M 3 0
L and ll in function M 3 1
L and ll in function M 3 2
L and ll in function M 3 3


INFO:tapm.HIVandSTI.model_HIVandSTI:Using exponential function to calculate m
INFO:tapm.HIVandSTI.model_HIVandSTI:Parameters: min_exp = 0.0, max_exp = 1.0, tau_exp = 0.2


L and ll in function M 0 0
L and ll in function M 0 1
L and ll in function M 0 2
L and ll in function M 0 3
L and ll in function M 1 0
L and ll in function M 1 1
L and ll in function M 1 2
L and ll in function M 1 3
L and ll in function M 2 0
L and ll in function M 2 1
L and ll in function M 2 2
L and ll in function M 2 3
L and ll in function M 3 0
L and ll in function M 3 1
L and ll in function M 3 2
L and ll in function M 3 3
L and ll in function M 0 0
L and ll in function M 0 1
L and ll in function M 0 2
L and ll in function M 0 3
L and ll in function M 1 0
L and ll in function M 1 1
L and ll in function M 1 2
L and ll in function M 1 3
L and ll in function M 2 0
L and ll in function M 2 1
L and ll in function M 2 2
L and ll in function M 2 3
L and ll in function M 3 0
L and ll in function M 3 1
L and ll in function M 3 2
L and ll in function M 3 3
L and ll in function M 0 0
L and ll in function M 0 1
L and ll in function M 0 2
L and ll in function M 0 3
L and ll in function M 1 0
L

# Only run untill the above. ToDo: delete the code below this markdown

In [ ]:
 
# Parameters from GannaRozhnova's paper (Elimination prospects of the Dutch HIV epidemic among men who have sex with men in the era of pre-exposure prophylaxis)
N0 = 210000 # initial population size
N01 = 0.451*N0 # initial population size of risk group 1
N02 = 0.353*N0 # initial population size of risk group 2
N03 = 0.125*N0 # initial population size of risk group 3
N04 = 0.071*N0 # initial population size of risk group 4
N0s = [N01,N02,N03,N04]
mu = 1/45 # per year, rate of recruitment to sexually active population
Omega = 1-0.86 # PrEP effectiveness, baseline
c = [0.13,1.43,5.44,18.21] # per year, average number of partners in risk group l
h = [0.62,0.12,0.642,0.0] # infectivity of untreated individuals in stage k of infection
epsilon = 0.01 # infectivity of treated individuals
epsilonP = h[1]/2 # infectivity of MSM infected on PrEP
Lambda = 0.25 # transmission prob. per partnership
omega = 0.5 # mixing parameter, (0: assortative, 1: proportionate mixing)
Phi = -jnp.log(1-0.05) # per year, annual ART dropout rate
tau = -jnp.log(1-0.3) # per year, annual ART uptake rate
tauP1 = -jnp.log(1-0.95) # per year, annual ART uptake rate for MSM infected on PrEP
tauP2 = tauP1
tauP3 = tauP1
tauP4 = tauP1
tauPs = [tau,tauP1,tauP2,tauP3,tauP4]
rho1 = 1/0.142 # per year, rate of transition from stage 1 to 2 for untreated individuals
rho2 = 1/8.439 # per year, rate of transition from stage 2 to 3 for untreated individuals
rho3 = 1/1.184 # per year, rate of transition from stage 3 to 4 for untreated individuals
rho4 = 1/1.316 # per year, mortality rate for untreated individuals
rhos = [rho1,rho2,rho3,rho4]
gamma1 = 1/8.21 # per year, rate of transition from stage 1 to 2 for tretaed individuals
gamma2 = 1/54.0 # per year, rate of transition from stage 2 to 3 for treated individuals
gamma3 = 1/2.463 # per year, rate of transition from stage 3 to 4 for treated individuals
gamma4 = 1/2.737 # per year, mortality rate for treated individuals
gammas = [gamma1,gamma2,gamma3,gamma4]
Kon1 = -jnp.log(1-PrEPuptake_rg1) # annual PrEP uptake rate in risk group 1
Kon2 = -jnp.log(1-PrEPuptake_rg2) # annual PrEP uptake rate in risk group 2
Kon3 = -jnp.log(1-PrEPuptake_rg3) # annual PrEP uptake rate in risk group 3
Kon4 = -jnp.log(1-PrEPuptake_rg4) # annual PrEP uptake rate in risk group 4
Kons = [Kon1,Kon2,Kon3,Kon4]
Koff1 = 1/5.0 # per year, average duration of taing PrEP in risk group 1
Koff2 = Koff1
Koff3 = Koff1
Koff4 = Koff1
Koffs = [Koff1,Koff2,Koff3,Koff4]

args = dict(N0s=N0s, mu=mu, Omega=Omega, c=c, h=h, epsilon=epsilon, epsilonP=epsilonP, Lambda=Lambda, omega=omega, Phi=Phi, tau = tau, tauPs=tauPs, rhos=rhos, gammas=gammas, Kons=Kons, Koffs=Koffs)

def N(y): # number of people per risk group for a given state y (which means at a given time)
    N1 = np.sum([y["S1"],y["SP1"],y["I11"],y["IP11"],y["I12"],y["I13"],y["I14"],y["A11"],y["A12"],y["A13"],y["A14"]])
    N2 = np.sum([y["S2"],y["SP2"],y["I21"],y["IP21"],y["I22"],y["I23"],y["I24"],y["A21"],y["A22"],y["A23"],y["A24"]])
    N3 = np.sum([y["S3"],y["SP3"],y["I31"],y["IP31"],y["I32"],y["I33"],y["I34"],y["A31"],y["A32"],y["A33"],y["A34"]])
    N4 = np.sum([y["S4"],y["SP4"],y["I41"],y["IP41"],y["I42"],y["I43"],y["I44"],y["A41"],y["A42"],y["A43"],y["A44"]])
    return [N1,N2,N3,N4]

def M(l,ll,y,args): # mixing matrix
    c = args["c"] # unpack args we need
    omega = args["omega"]
    l = l-1 # because we start counting from 0
    ll = ll-1
    Ns = np.array(N(y))
    print("L and ll in function M", l,ll)
    return omega*c[ll]*Ns[ll]/(jnp.sum([c*Ns])) + (1-omega)*jnp.where(l == ll, 1, 0)

def JP(l,y,args): # force of infection per year in group l
    h = args["h"] # unpack args we need
    epsilon = args["epsilon"]
    epsilonP = args["epsilonP"]
    Lambda = args["Lambda"]
    c = args["c"]
    Is = np.array([[y["I11"], y["I12"], y["I13"], y["I14"]],[y["I21"], y["I22"], y["I23"], y["I24"]],[y["I31"], y["I32"], y["I33"], y["I34"]],[y["I41"], y["I42"], y["I43"], y["I44"]]])
    IPs = np.array([y["Ip11"], y["IP21"], y["IP31"], y["IP41"]])
    N = N(y)
    As = np.array([[y["A11"], y["A12"], y["A13"], y["A14"]],[y["A21"], y["A22"], y["A23"], y["A24"]],[y["A31"], y["A32"], y["A33"], y["A34"]],[y["A41"], y["A42"], y["A43"], y["A44"]]])
    Ms = np.array([M(l,1,y),M(l,2,y),M(l,3,y),M(l,4,y)])
    innersums = [jnp.sum(h*Is[ll]/N[ll] + epsilon*As[ll]/N[ll]) for ll in range(4)]
    sum = jnp.sum((Ms*(epsilonP*IPs/N + innersums)))
    JP = Lambda * c[l-1] * sum
    return JP

In [ ]:
# Parameters we have to decide
PrEPuptake_rg1 = 0.01 # annual PrEP uptake in risk group 1 (fraction)
PrEPuptake_rg2 = 0.025 # annual PrEP uptake in risk group 2 (fraction)
PrEPuptake_rg3 = 0.05 # annual PrEP uptake in risk group 3 (fraction)
PrEPuptake_rg4 = 0.1 # annual PrEP uptake in risk group 4 (fraction)

#Starting values -- The total number per risk group has to amount to the starting values N0j (see above)
# Susceptible per risk group
S1_0 = 0.58*N01
S2_0 =  0.58*N02
S3_0 =  0.58*N03
S4_0 =  0.58*N04
# Susceptible on PrEP per risk group
SP1_0 = 0.01*N01
SP2_0 = 0.01*N02
SP3_0 = 0.01*N03
SP4_0 = 0.01*N04
# Infected in stage 1 per risk group
I11_0 = 0.05*N01
I21_0 = 0.05*N02
I31_0 = 0.05*N03
I41_0 = 0.05*N04
# Infected on PrEP in stage 1 per risk group
IP11_0 = 0.01*N01
IP21_0 = 0.01*N02
IP31_0 = 0.01*N03
IP41_0 = 0.01*N04
# Infected in stage 2 per risk group
I12_0 = 0.05*N01
I22_0 = 0.05*N02
I32_0 = 0.05*N03
I42_0 = 0.05*N04
# Infected in stage 3 per risk group
I13_0 = 0.05*N01
I23_0 = 0.05*N02
I33_0 = 0.05*N03
I43_0 = 0.05*N04
# Infected in stage 4 per risk group
I14_0 = 0.05*N01
I24_0 = 0.05*N02
I34_0 = 0.05*N03
I44_0 = 0.05*N04
# ART in stage 1 per risk group
A11_0 = 0.05*N01
A21_0 = 0.05*N02
A31_0 = 0.05*N03
A41_0 = 0.05*N04
# ART in stage 2 per risk group
A12_0 = 0.05*N01
A22_0 = 0.05*N02
A32_0 = 0.05*N03
A42_0 = 0.05*N04
# ART in stage 3 per risk group
A13_0 = 0.05*N01
A23_0 = 0.05*N02
A33_0 = 0.05*N03
A43_0 = 0.05*N04
# ART in stage 4 per risk group
A14_0 = 0.05*N01
A24_0 = 0.05*N02
A34_0 = 0.05*N03
A44_0 = 0.05*N04


In [ ]:
# Equations

def HIV_model(t, y, args): # model from Ganna Rozhnova
    cm = icomo.CompModel(y)  # Initialize the compartmental model

    # unpack args
    Kons, N0s, mu, Omega, Koffs, tau, rhos, tauPs, Phi, gammas = args['Kons'], args['N0s'], args['mu'], args['Omega'], args['Koffs'], args['tau'], args['rhos'], args['tauPs'], args['Phi'], args['gammas']

    # Flow out of S compartments
    cm.flow("S1", "I11",  JP(1,cm.y,args))
    cm.flow("S2", "I21",  JP(2,cm.y,args))
    cm.flow("S3", "I31",  JP(3,cm.y,args))
    cm.flow("S4", "I41",  JP(4,cm.y,args))
    cm.flow("S1", "SP1", Kons[0])
    cm.flow("S2", "SP2", Kons[1])
    cm.flow("S3", "SP3", Kons[2])
    cm.flow("S4", "SP4", Kons[3])
    cm.dy["S1"] = cm.dy["S1"] - mu*cm.y["S1"] + mu*N0s[0]
    cm.dy["S2"] = cm.dy["S2"] - mu*cm.y["S2"] + mu*N0s[1]
    cm.dy["S3"] = cm.dy["S3"] - mu*cm.y["S3"] + mu*N0s[2]
    cm.dy["S4"] = cm.dy["S4"] - mu*cm.y["S4"] + mu*N0s[3]

    # Flow out of SP compartments
    cm.flow("SP1", "IP11", Omega*JP(1,cm.y,args))
    cm.flow("SP2", "IP21", Omega*JP(2,cm.y,args))
    cm.flow("SP3", "IP31", Omega*JP(3,cm.y,args))
    cm.flow("SP4", "IP41", Omega*JP(4,cm.y,args))
    cm.flow("SP1", "S1", Koffs[0])
    cm.flow("SP2", "S2", Koffs[1])
    cm.flow("SP3", "S3", Koffs[2])
    cm.flow("SP4", "S4", Koffs[3])
    cm.dy["SP1"] = cm.dy["SP1"] - mu*cm.y["SP1"]
    cm.dy["SP2"] = cm.dy["SP2"] - mu*cm.y["SP2"]
    cm.dy["SP3"] = cm.dy["SP3"] - mu*cm.y["SP3"]
    cm.dy["SP4"] = cm.dy["SP4"] - mu*cm.y["SP4"]

    # Flow out of Il1 compartments
    cm.flow("I11", "A11", tau)
    cm.flow("I21", "A21", tau)
    cm.flow("I31", "A31", tau)
    cm.flow("I41", "A41", tau)
    cm.dy["I11"] = cm.dy["I11"] - (mu + rhos[0])*cm.y["I11"]
    cm.dy["I21"] = cm.dy["I21"] - (mu + rhos[0])*cm.y["I21"]
    cm.dy["I31"] = cm.dy["I31"] - (mu + rhos[0])*cm.y["I31"]
    cm.dy["I41"] = cm.dy["I41"] - (mu + rhos[0])*cm.y["I41"]

    # Flow out of IPl1 compartments
    cm.flow("IP11", "A11", tauPs[0])
    cm.flow("IP21", "A21", tauPs[1])
    cm.flow("IP31", "A31", tauPs[2])
    cm.flow("IP41", "A41", tauPs[3])
    cm.dy["IP11"] = cm.dy["IP11"] - mu*cm.y["IP11"]
    cm.dy["IP21"] = cm.dy["IP21"] - mu*cm.y["IP21"]
    cm.dy["IP31"] = cm.dy["IP31"] - mu*cm.y["IP31"]
    cm.dy["IP41"] = cm.dy["IP41"] - mu*cm.y["IP41"]

    # Flow out of Ilk compartments (k=2,3,4)
    # k=2
    cm.flow("I12", "A12", tau)
    cm.flow("I22", "A22", tau)
    cm.flow("I32", "A32", tau)  
    cm.flow("I42", "A42", tau)
    cm.dy["I12"] = cm.dy["I12"] - (mu + rhos[1])*cm.y["I12"] + rhos[0]*cm.y["I11"]
    cm.dy["I22"] = cm.dy["I22"] - (mu + rhos[1])*cm.y["I22"] + rhos[0]*cm.y["I21"]
    cm.dy["I32"] = cm.dy["I32"] - (mu + rhos[1])*cm.y["I32"] + rhos[0]*cm.y["I31"]
    cm.dy["I42"] = cm.dy["I42"] - (mu + rhos[1])*cm.y["I42"] + rhos[0]*cm.y["I41"]
    # k=3
    cm.flow("I13", "A13", tau)
    cm.flow("I23", "A23", tau)
    cm.flow("I33", "A33", tau)
    cm.flow("I43", "A43", tau)
    cm.dy["I13"] = cm.dy["I13"] - (mu + rhos[2])*cm.y["I13"] + rhos[1]*cm.y["I12"]
    cm.dy["I23"] = cm.dy["I23"] - (mu + rhos[2])*cm.y["I23"] + rhos[1]*cm.y["I22"]
    cm.dy["I33"] = cm.dy["I33"] - (mu + rhos[2])*cm.y["I33"] + rhos[1]*cm.y["I32"]
    cm.dy["I43"] = cm.dy["I43"] - (mu + rhos[2])*cm.y["I43"] + rhos[1]*cm.y["I42"]
    # k=4
    cm.flow("I14", "A14", tau)
    cm.flow("I24", "A24", tau)
    cm.flow("I34", "A34", tau)
    cm.flow("I44", "A44", tau)
    cm.dy["I14"] = cm.dy["I14"] - (mu + rhos[3])*cm.y["I14"] + rhos[2]*cm.y["I13"]
    cm.dy["I24"] = cm.dy["I24"] - (mu + rhos[3])*cm.y["I24"] + rhos[2]*cm.y["I23"]
    cm.dy["I34"] = cm.dy["I34"] - (mu + rhos[3])*cm.y["I34"] + rhos[2]*cm.y["I33"]
    cm.dy["I44"] = cm.dy["I44"] - (mu + rhos[3])*cm.y["I44"] + rhos[2]*cm.y["I43"]

    # Flow out of Al1 compartments
    cm.flow("A11", "I11", Phi)
    cm.flow("A21", "I21", Phi)
    cm.flow("A31", "I31", Phi)
    cm.flow("A41", "I41", Phi)
    cm.dy["A11"] = cm.dy["A11"] - (mu + gammas[0])*cm.y["A11"]
    cm.dy["A21"] = cm.dy["A21"] - (mu + gammas[0])*cm.y["A21"]
    cm.dy["A31"] = cm.dy["A31"] - (mu + gammas[0])*cm.y["A31"]
    cm.dy["A41"] = cm.dy["A41"] - (mu + gammas[0])*cm.y["A41"]

    # Flow out of Alk compartments (k=2,3,4)
    # k=2
    cm.flow("A12", "I12", Phi)
    cm.flow("A22", "I22", Phi)
    cm.flow("A32", "I32", Phi)
    cm.flow("A42", "I42", Phi)
    cm.dy["A12"] = cm.dy["A12"] - (mu + gammas[1])*cm.y["A12"] + gammas[0]*cm.y["A11"]
    cm.dy["A22"] = cm.dy["A22"] - (mu + gammas[1])*cm.y["A22"] + gammas[0]*cm.y["A21"]
    cm.dy["A32"] = cm.dy["A32"] - (mu + gammas[1])*cm.y["A32"] + gammas[0]*cm.y["A31"]
    cm.dy["A42"] = cm.dy["A42"] - (mu + gammas[1])*cm.y["A42"] + gammas[0]*cm.y["A41"]
    # k=3
    cm.flow("A13", "I13", Phi)
    cm.flow("A23", "I23", Phi)
    cm.flow("A33", "I33", Phi)
    cm.flow("A43", "I43", Phi)
    cm.dy["A13"] = cm.dy["A13"] - (mu + gammas[2])*cm.y["A13"] + gammas[1]*cm.y["A12"]
    cm.dy["A23"] = cm.dy["A23"] - (mu + gammas[2])*cm.y["A23"] + gammas[1]*cm.y["A22"]
    cm.dy["A33"] = cm.dy["A33"] - (mu + gammas[2])*cm.y["A33"] + gammas[1]*cm.y["A32"]
    cm.dy["A43"] = cm.dy["A43"] - (mu + gammas[2])*cm.y["A43"] + gammas[1]*cm.y["A42"]
    # k=4
    cm.flow("A14", "I14", Phi)
    cm.flow("A24", "I24", Phi)
    cm.flow("A34", "I34", Phi)
    cm.flow("A44", "I44", Phi)
    cm.dy["A14"] = cm.dy["A14"] - (mu + gammas[3])*cm.y["A14"] + gammas[2]*cm.y["A13"]
    cm.dy["A24"] = cm.dy["A24"] - (mu + gammas[3])*cm.y["A24"] + gammas[2]*cm.y["A23"]
    cm.dy["A34"] = cm.dy["A34"] - (mu + gammas[3])*cm.y["A34"] + gammas[2]*cm.y["A33"]
    cm.dy["A44"] = cm.dy["A44"] - (mu + gammas[3])*cm.y["A44"] + gammas[2]*cm.y["A43"]

    # Return the differential changes
    return cm.dy


# Function to setup the model and return the integrator
def setup_model():

    # Define the time span for the simulation
    ts = np.linspace(0, 3600 * 5, 3600)

    # Create an ODE integrator object using the icomo library
    integrator_object = icomo.ODEIntegrator(
        ts_out=ts,  # Output time points
        t_0=min(ts),  # Initial time point
        ts_solver=ts,  # Time points for the solver to use
        ts_arg=ts,
    )

    # Get the integration function for the model
    integrator = integrator_object.get_func(HIV_model)  # Returns a function that can be used to solve the ODEs defined in the 'model' function


    return integrator






In [ ]:
integrator = setup_model()
output = output = integrator(y0=y0, constant_args=args)

# STI MODEL

In [ ]:
# This python file contains the model definition for the STI dynamics in the model.
# The model is defined as a set of differential equations that describe the flow of individuals between different compartments.
# The model includes compartments for susceptible individuals, asymptomatic STI cases, symptomatic STI cases, and individuals undergoing treatment for STI.
# The model also includes parameters that describe the transmission dynamics of the STI, the testing and treatment rates, and the impact of HIV on STI transmission.
# The model is implemented using the icomo library, which provides tools for defining and solving compartmental models. The model is set up to be integrated over a time span of 5 years, with output time points every hour.
# The model is ready for simulation once it is set up.

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import jax
import jax.numpy as jnp
import icomo
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Global flag to track logging
logged_exp_logis = False


def m(args, H):
    """
    Exponential function with three parameters: minimum value, maximum value, and rate/tau.

    Args:
    args (dict): A dictionary containing the parameters 'H', 'min_exp', 'max_exp', and 'tau_exp'.
    H (float): The current value of 'H'.

    Returns:
    float: The output of the exponential function.
    """
    global logged_exp_logis
    logger.debug(
        "Calculating self-regulation factor factor 'm' using exponential function"
    )
    if H is None:
        H = args["H"]
    min_exp = args["min_exp"]
    max_exp = args["max_exp"]
    tau_exp = args["tau_exp"] * args["scaling_factor_m_eps"]

    if not logged_exp_logis:
        logger.info("Using exponential function to calculate m")
        logger.info(
            "Parameters: min_exp = %s, max_exp = %s, tau_exp = %s",
            min_exp,
            max_exp,
            tau_exp,
        )
        logged_exp_logis = True

    return min_exp + (max_exp - min_exp) * (1 - jnp.exp(-H / tau_exp))


def Ml1_first_term(y, args):
    c = args["c"] # unpack args we need
    omega = args["omega"]
    Ns = np.array(N(y))
    return omega*c[0]*Ns[0]/(jnp.sum([c*Ns]))

def Ml2_first_term(y, args):
    c = args["c"] # unpack args we need
    omega = args["omega"]
    Ns = np.array(N(y))
    return omega*c[1]*Ns[1]/(jnp.sum([c*Ns]))

def Ml3_first_term(y, args):
    c = args["c"] # unpack args we need
    omega = args["omega"]
    Ns = np.array(N(y))
    return omega*c[2]*Ns[2]/(jnp.sum([c*Ns]))

def Ml4_first_term(y, args):
    c = args["c"] # unpack args we need
    omega = args["omega"]
    Ns = np.array(N(y))
    return omega*c[3]*Ns[3]/(jnp.sum([c*Ns]))

def J1(y, args):
    beta_0_sti = args["beta_STI"]
    c = args["c"]
    M_second_terms = [1-omega,0,0,0]
    Is = np.array([y["Is1"],y["Is2"],y["Is3"],y["Is4"]])
    Ia = np.array([y["Ia1"],y["Ia2"],y["Ia3"],y["Ia4"]])
    N = N(y)
    Ms = np.array([Ml1_first_term(y, args),Ml2_first_term(y, args),Ml3_first_term(y, args),Ml4_first_term(y, args)])
    innersums = [jnp.sum(beta_0_sti*(Is[ll]+Ia[ll])/N[ll]) for ll in range(4)]
    sum = jnp.sum((jnp.add(Ms,M_second_terms)*(innersums)))
    JP = Lambda * c[0] * sum
    return JP

def J2(y, args):
    beta_0_sti = args["beta_STI"]
    c = args["c"]
    M_second_terms = [0,1-omega,0,0]
    Is = np.array([y["Is1"],y["Is2"],y["Is3"],y["Is4"]])
    Ia = np.array([y["Ia1"],y["Ia2"],y["Ia3"],y["Ia4"]])
    N = N(y)
    Ms = np.array([Ml1_first_term(y, args),Ml2_first_term(y, args),Ml3_first_term(y, args),Ml4_first_term(y, args)])
    innersums = [jnp.sum(beta_0_sti*(Is[ll]+Ia[ll])/N[ll]) for ll in range(4)]
    sum = jnp.sum((jnp.add(Ms,M_second_terms)*(innersums)))
    JP = Lambda * c[1] * sum
    return JP

def J3(y, args):
    beta_0_sti = args["beta_STI"]
    c = args["c"]
    M_second_terms = [0,0,1-omega,0]
    Is = np.array([y["Is1"],y["Is2"],y["Is3"],y["Is4"]])
    Ia = np.array([y["Ia1"],y["Ia2"],y["Ia3"],y["Ia4"]])
    N = N(y)
    Ms = np.array([Ml1_first_term(y, args),Ml2_first_term(y, args),Ml3_first_term(y, args),Ml4_first_term(y, args)])
    innersums = [jnp.sum(beta_0_sti*(Is[ll]+Ia[ll])/N[ll]) for ll in range(4)]
    sum = jnp.sum((jnp.add(Ms,M_second_terms)*(innersums)))
    JP = Lambda * c[2] * sum
    return JP

def J4(y, args):
    beta_0_sti = args["beta_STI"]
    c = args["c"]
    M_second_terms = [0,0,0,1-omega]
    Is = np.array([y["Is1"],y["Is2"],y["Is3"],y["Is4"]])
    Ia = np.array([y["Ia1"],y["Ia2"],y["Ia3"],y["Ia4"]])
    N = N(y)
    Ms = np.array([Ml1_first_term(y, args),Ml2_first_term(y, args),Ml3_first_term(y, args),Ml4_first_term(y, args)])
    innersums = [jnp.sum(beta_0_sti*(Is[ll]+Ia[ll])/N[ll]) for ll in range(4)]
    sum = jnp.sum((jnp.add(Ms,M_second_terms)*(innersums)))
    JP = Lambda * c[3] * sum
    return JP
    
# Function to calculate the testing rate of STI
def lambda_a(args):
    """
    Calculate the testing rate of assymptomatic STI.

    Args:
        args (dict): A dictionary containing the following parameters:
            - lambda_0 (float): Baseline test rate
            - c (float): Constant term
            - m (function): Function to calculate m value
            - beta_HIV (float): HIV transmission rate
            - H (float): Number of susceptible individuals
            - P_HIV (float): HIV prevalence

    Returns:
        float: The testing rate of STI.

    """
    logger.debug("Calculating testing rate of STI")
    return (
        args["lambda_0"]  # Baseline test rate
        + args["c"]
        * (1 - m(args, H=None))
        * args["beta_HIV"]
        * args["H"]
        * (1 - args["P_HIV"])  # HIV dependent term
        + args["lambda_P"]
        * args["P_HIV"]  # Proportional infection rate due to HIV prevalence
    )


# Function to calculate infection from asymptomatic STI individuals
def infect_ia1(y, args):
    logger.debug("Calculating infection from asymptomatic STI individuals")
    return (
        (args["asymptomatic"])
        * (1 - m(args, H=None) * (1 - args["P_HIV"]))
        * J1(y, args)
    )


# Function to calculate infection from symptomatic STI individuals
def infect_is1(y, args):
    logger.debug("Calculating infection from symptomatic STI individuals")
    return (
        (1 - args["asymptomatic"])
        * (1 - m(args, H=None) * (1 - args["P_HIV"]))
        * J1(y, args)
    )

def infect_ia2(y, args):
    logger.debug("Calculating infection from asymptomatic STI individuals")
    return (
        (args["asymptomatic"])
        * (1 - m(args, H=None) * (1 - args["P_HIV"]))
        * J2(y, args)
    )


# Function to calculate infection from symptomatic STI individuals
def infect_is2(y, args):
    logger.debug("Calculating infection from symptomatic STI individuals")
    return (
        (1 - args["asymptomatic"])
        * (1 - m(args, H=None) * (1 - args["P_HIV"]))
        * J2(y, args)
    )

def infect_ia3(y, args):
    logger.debug("Calculating infection from asymptomatic STI individuals")
    return (
        (args["asymptomatic"])
        * (1 - m(args, H=None) * (1 - args["P_HIV"]))
        * J3(y, args)
    )


# Function to calculate infection from symptomatic STI individuals
def infect_is3(y, args):
    logger.debug("Calculating infection from symptomatic STI individuals")
    return (
        (1 - args["asymptomatic"])
        * (1 - m(args, H=None) * (1 - args["P_HIV"]))
        * J3(y, args)
    )

def infect_ia4(y, args):
    logger.debug("Calculating infection from asymptomatic STI individuals")
    return (
        (args["asymptomatic"])
        * (1 - m(args, H=None) * (1 - args["P_HIV"]))
        * J4(y, args)
    )


# Function to calculate infection from symptomatic STI individuals
def infect_is4(y, args):
    logger.debug("Calculating infection from symptomatic STI individuals")
    return (
        (1 - args["asymptomatic"])
        * (1 - m(args, H=None) * (1 - args["P_HIV"]))
        * J4(y, args)
    )


# Main model function that defines the differential equations of the system
def model_STI(t, y, args):
    """
    Calculate the differential changes in a compartmental model for STI dynamics.

    Parameters:
    t (float): The time at which the model is evaluated.
    y (list): The current values of the compartments in the model.
    args (dict): Additional arguments required for the model calculations.

    Returns:
    list: The differential changes in the compartments.

    """
    logger.debug("Defining the differential equations of the system")
    cm = icomo.CompModel(y)  # Initialize the compartmental model

    # Basic STI dynamics
    cm.flow("S1_STI", "Ia1_STI", infect_ia1(y, args))  # Susceptible to asymptomatic
    cm.flow("S1_STI", "Is1_STI", infect_is1(y, args))  # Susceptible to symptomatic
    cm.flow("Ia1_STI", "S1_STI", args["gamma_STI"])  # Asymptomatic to susceptible (recovery)
    cm.flow("Ia1_STI", "T1_STI", lambda_a(args))  # Asymptomatic to tested and treatment
    cm.flow("Is1_STI", "T1_STI", args["lambda_s"])  # Symptomatic to tested and treatment
    cm.flow("T1_STI", "S1_STI", args["gammaT_STI"])  # Treatment to susceptible (immunity loss)

    # Vital dynamics (New addition/removoval to/from risk group)
    cm.dy["S1_STI"] = cm.dy["S1_STI"] - args["mu"] * cm.y["S1_STI"] + args["mu"] * args["N0s"][0]
    cm.dy["Ia1_STI"] = cm.dy["Ia1_STI"] - args["mu"] * cm.y["Ia1_STI"]
    cm.dy["Is1_STI"] = cm.dy["Is1_STI"] - args["mu"] * cm.y["Is1_STI"] 
    cm.dy["T1_STI"] = cm.dy["T1_STI"] - args["mu"] * cm.y["T1_STI"]

    cm.flow("S2_STI", "Ia2_STI", infect_ia2(y, args))  # Susceptible to asymptomatic
    cm.flow("S2_STI", "Is2_STI", infect_is2(y, args))  # Susceptible to symptomatic
    cm.flow("Ia2_STI", "S2_STI", args["gamma_STI"])  # Asymptomatic to susceptible (recovery)
    cm.flow("Ia2_STI", "T2_STI", lambda_a(args))  # Asymptomatic to tested and treatment
    cm.flow("Is2_STI", "T2_STI", args["lambda_s"])  # Symptomatic to tested and treatment
    cm.flow("T2_STI", "S2_STI", args["gammaT_STI"])  # Treatment to susceptible (immunity loss)

    # Vital dynamics (New addition/removal to/from risk group)
    cm.dy["S2_STI"] = cm.dy["S2_STI"] - args["mu"] * cm.y["S2_STI"] + args["mu"] * args["N0s"][1]
    cm.dy["Ia2_STI"] = cm.dy["Ia2_STI"] - args["mu"] * cm.y["Ia2_STI"]
    cm.dy["Is2_STI"] = cm.dy["Is2_STI"] - args["mu"] * cm.y["Is2_STI"]
    cm.dy["T2_STI"] = cm.dy["T2_STI"] - args["mu"] * cm.y["T2_STI"]

    cm.flow("S3_STI", "Ia3_STI", infect_ia3(y, args))  # Susceptible to asymptomatic
    cm.flow("S3_STI", "Is3_STI", infect_is3(y, args))  # Susceptible to symptomatic
    cm.flow("Ia3_STI", "S3_STI", args["gamma_STI"])  # Asymptomatic to susceptible (recovery)
    cm.flow("Ia3_STI", "T3_STI", lambda_a(args))  # Asymptomatic to tested and treatment
    cm.flow("Is3_STI", "T3_STI", args["lambda_s"])  # Symptomatic to tested and treatment
    cm.flow("T3_STI", "S3_STI", args["gammaT_STI"])  # Treatment to susceptible (immunity loss)

    # Vital dynamics (New addition/removal to/from risk group)
    cm.dy["S3_STI"] = cm.dy["S3_STI"] - args["mu"] * cm.y["S3_STI"] + args["mu"] * args["N0s"][2]
    cm.dy["Ia3_STI"] = cm.dy["Ia3_STI"] - args["mu"] * cm.y["Ia3_STI"]
    cm.dy["Is3_STI"] = cm.dy["Is3_STI"] - args["mu"] * cm.y["Is3_STI"]
    cm.dy["T3_STI"] = cm.dy["T3_STI"] - args["mu"] * cm.y["T3_STI"]

    cm.flow("S4_STI", "Ia4_STI", infect_ia4(y, args))  # Susceptible to asymptomatic
    cm.flow("S4_STI", "Is4_STI", infect_is4(y, args))  # Susceptible to symptomatic
    cm.flow("Ia4_STI", "S4_STI", args["gamma_STI"])  # Asymptomatic to susceptible (recovery)
    cm.flow("Ia4_STI", "T4_STI", lambda_a(args))  # Asymptomatic to tested and treatment
    cm.flow("Is4_STI", "T4_STI", args["lambda_s"])  # Symptomatic to tested and treatment
    cm.flow("T4_STI", "S4_STI", args["gammaT_STI"])  # Treatment to susceptible (immunity loss)

    # Vital dynamics (New addition/removal to/from risk group)
    cm.dy["S4_STI"] = cm.dy["S4_STI"] - args["mu"] * cm.y["S4_STI"] + args["mu"] * args["N0s"][3]
    cm.dy["Ia4_STI"] = cm.dy["Ia4_STI"] - args["mu"] * cm.y["Ia4_STI"]
    cm.dy["Is4_STI"] = cm.dy["Is4_STI"] - args["mu"] * cm.y["Is4_STI"]
    cm.dy["T4_STI"] = cm.dy["T4_STI"] - args["mu"] * cm.y["T4_STI"]

    # Return the differential changes
    return cm.dy

def main_model(t, y, args):
    cm = icomo.CompModel(y)  # Initialize the compartmental model

    # unpack args
    Kons, N0s, mu, Omega, Koffs, tau, rhos, tauPs, Phi, gammas = args['Kons'], args['N0s'], args['mu'], args['Omega'], args['Koffs'], args['tau'], args['rhos'], args['tauPs'], args['Phi'], args['gammas']

    # HIV dynamics
    # Flow out of S compartments
    cm.flow("S1", "I11",  JP(1,cm.y,args))
    cm.flow("S2", "I21",  JP(2,cm.y,args))
    cm.flow("S3", "I31",  JP(3,cm.y,args))
    cm.flow("S4", "I41",  JP(4,cm.y,args))
    cm.flow("S1", "SP1", Kons[0])
    cm.flow("S2", "SP2", Kons[1])
    cm.flow("S3", "SP3", Kons[2])
    cm.flow("S4", "SP4", Kons[3])
    cm.dy["S1"] = cm.dy["S1"] - mu*cm.y["S1"] + mu*N0s[0]
    cm.dy["S2"] = cm.dy["S2"] - mu*cm.y["S2"] + mu*N0s[1]
    cm.dy["S3"] = cm.dy["S3"] - mu*cm.y["S3"] + mu*N0s[2]
    cm.dy["S4"] = cm.dy["S4"] - mu*cm.y["S4"] + mu*N0s[3]

    # Flow out of SP compartments
    cm.flow("SP1", "IP11", Omega*JP(1,cm.y,args))
    cm.flow("SP2", "IP21", Omega*JP(2,cm.y,args))
    cm.flow("SP3", "IP31", Omega*JP(3,cm.y,args))
    cm.flow("SP4", "IP41", Omega*JP(4,cm.y,args))
    cm.flow("SP1", "S1", Koffs[0])
    cm.flow("SP2", "S2", Koffs[1])
    cm.flow("SP3", "S3", Koffs[2])
    cm.flow("SP4", "S4", Koffs[3])
    cm.dy["SP1"] = cm.dy["SP1"] - mu*cm.y["SP1"]
    cm.dy["SP2"] = cm.dy["SP2"] - mu*cm.y["SP2"]
    cm.dy["SP3"] = cm.dy["SP3"] - mu*cm.y["SP3"]
    cm.dy["SP4"] = cm.dy["SP4"] - mu*cm.y["SP4"]

    # Flow out of Il1 compartments
    cm.flow("I11", "A11", tau)
    cm.flow("I21", "A21", tau)
    cm.flow("I31", "A31", tau)
    cm.flow("I41", "A41", tau)
    cm.dy["I11"] = cm.dy["I11"] - (mu + rhos[0])*cm.y["I11"]
    cm.dy["I21"] = cm.dy["I21"] - (mu + rhos[0])*cm.y["I21"]
    cm.dy["I31"] = cm.dy["I31"] - (mu + rhos[0])*cm.y["I31"]
    cm.dy["I41"] = cm.dy["I41"] - (mu + rhos[0])*cm.y["I41"]

    # Flow out of IPl1 compartments
    cm.flow("IP11", "A11", tauPs[0])
    cm.flow("IP21", "A21", tauPs[1])
    cm.flow("IP31", "A31", tauPs[2])
    cm.flow("IP41", "A41", tauPs[3])
    cm.dy["IP11"] = cm.dy["IP11"] - mu*cm.y["IP11"]
    cm.dy["IP21"] = cm.dy["IP21"] - mu*cm.y["IP21"]
    cm.dy["IP31"] = cm.dy["IP31"] - mu*cm.y["IP31"]
    cm.dy["IP41"] = cm.dy["IP41"] - mu*cm.y["IP41"]

    # Flow out of Ilk compartments (k=2,3,4)
    # k=2
    cm.flow("I12", "A12", tau)
    cm.flow("I22", "A22", tau)
    cm.flow("I32", "A32", tau)  
    cm.flow("I42", "A42", tau)
    cm.dy["I12"] = cm.dy["I12"] - (mu + rhos[1])*cm.y["I12"] + rhos[0]*cm.y["I11"]
    cm.dy["I22"] = cm.dy["I22"] - (mu + rhos[1])*cm.y["I22"] + rhos[0]*cm.y["I21"]
    cm.dy["I32"] = cm.dy["I32"] - (mu + rhos[1])*cm.y["I32"] + rhos[0]*cm.y["I31"]
    cm.dy["I42"] = cm.dy["I42"] - (mu + rhos[1])*cm.y["I42"] + rhos[0]*cm.y["I41"]
    # k=3
    cm.flow("I13", "A13", tau)
    cm.flow("I23", "A23", tau)
    cm.flow("I33", "A33", tau)
    cm.flow("I43", "A43", tau)
    cm.dy["I13"] = cm.dy["I13"] - (mu + rhos[2])*cm.y["I13"] + rhos[1]*cm.y["I12"]
    cm.dy["I23"] = cm.dy["I23"] - (mu + rhos[2])*cm.y["I23"] + rhos[1]*cm.y["I22"]
    cm.dy["I33"] = cm.dy["I33"] - (mu + rhos[2])*cm.y["I33"] + rhos[1]*cm.y["I32"]
    cm.dy["I43"] = cm.dy["I43"] - (mu + rhos[2])*cm.y["I43"] + rhos[1]*cm.y["I42"]
    # k=4
    cm.flow("I14", "A14", tau)
    cm.flow("I24", "A24", tau)
    cm.flow("I34", "A34", tau)
    cm.flow("I44", "A44", tau)
    cm.dy["I14"] = cm.dy["I14"] - (mu + rhos[3])*cm.y["I14"] + rhos[2]*cm.y["I13"]
    cm.dy["I24"] = cm.dy["I24"] - (mu + rhos[3])*cm.y["I24"] + rhos[2]*cm.y["I23"]
    cm.dy["I34"] = cm.dy["I34"] - (mu + rhos[3])*cm.y["I34"] + rhos[2]*cm.y["I33"]
    cm.dy["I44"] = cm.dy["I44"] - (mu + rhos[3])*cm.y["I44"] + rhos[2]*cm.y["I43"]

    # Flow out of Al1 compartments
    cm.flow("A11", "I11", Phi)
    cm.flow("A21", "I21", Phi)
    cm.flow("A31", "I31", Phi)
    cm.flow("A41", "I41", Phi)
    cm.dy["A11"] = cm.dy["A11"] - (mu + gammas[0])*cm.y["A11"]
    cm.dy["A21"] = cm.dy["A21"] - (mu + gammas[0])*cm.y["A21"]
    cm.dy["A31"] = cm.dy["A31"] - (mu + gammas[0])*cm.y["A31"]
    cm.dy["A41"] = cm.dy["A41"] - (mu + gammas[0])*cm.y["A41"]

    # Flow out of Alk compartments (k=2,3,4)
    # k=2
    cm.flow("A12", "I12", Phi)
    cm.flow("A22", "I22", Phi)
    cm.flow("A32", "I32", Phi)
    cm.flow("A42", "I42", Phi)
    cm.dy["A12"] = cm.dy["A12"] - (mu + gammas[1])*cm.y["A12"] + gammas[0]*cm.y["A11"]
    cm.dy["A22"] = cm.dy["A22"] - (mu + gammas[1])*cm.y["A22"] + gammas[0]*cm.y["A21"]
    cm.dy["A32"] = cm.dy["A32"] - (mu + gammas[1])*cm.y["A32"] + gammas[0]*cm.y["A31"]
    cm.dy["A42"] = cm.dy["A42"] - (mu + gammas[1])*cm.y["A42"] + gammas[0]*cm.y["A41"]
    # k=3
    cm.flow("A13", "I13", Phi)
    cm.flow("A23", "I23", Phi)
    cm.flow("A33", "I33", Phi)
    cm.flow("A43", "I43", Phi)
    cm.dy["A13"] = cm.dy["A13"] - (mu + gammas[2])*cm.y["A13"] + gammas[1]*cm.y["A12"]
    cm.dy["A23"] = cm.dy["A23"] - (mu + gammas[2])*cm.y["A23"] + gammas[1]*cm.y["A22"]
    cm.dy["A33"] = cm.dy["A33"] - (mu + gammas[2])*cm.y["A33"] + gammas[1]*cm.y["A32"]
    cm.dy["A43"] = cm.dy["A43"] - (mu + gammas[2])*cm.y["A43"] + gammas[1]*cm.y["A42"]
    # k=4
    cm.flow("A14", "I14", Phi)
    cm.flow("A24", "I24", Phi)
    cm.flow("A34", "I34", Phi)
    cm.flow("A44", "I44", Phi)
    cm.dy["A14"] = cm.dy["A14"] - (mu + gammas[3])*cm.y["A14"] + gammas[2]*cm.y["A13"]
    cm.dy["A24"] = cm.dy["A24"] - (mu + gammas[3])*cm.y["A24"] + gammas[2]*cm.y["A23"]
    cm.dy["A34"] = cm.dy["A34"] - (mu + gammas[3])*cm.y["A34"] + gammas[2]*cm.y["A33"]
    cm.dy["A44"] = cm.dy["A44"] - (mu + gammas[3])*cm.y["A44"] + gammas[2]*cm.y["A43"]

    ## STI dynamics
        # Basic STI dynamics
    cm.flow("S1_STI", "Ia1_STI", infect_ia1(y, args))  # Susceptible to asymptomatic
    cm.flow("S1_STI", "Is1_STI", infect_is1(y, args))  # Susceptible to symptomatic
    cm.flow("Ia1_STI", "S1_STI", args["gamma_STI"])  # Asymptomatic to susceptible (recovery)
    cm.flow("Ia1_STI", "T1_STI", lambda_a(args))  # Asymptomatic to tested and treatment
    cm.flow("Is1_STI", "T1_STI", args["lambda_s"])  # Symptomatic to tested and treatment
    cm.flow("T1_STI", "S1_STI", args["gammaT_STI"])  # Treatment to susceptible (immunity loss)

    # Vital dynamics (New addition/removoval to/from risk group)
    cm.dy["S1_STI"] = cm.dy["S1_STI"] - args["mu"] * cm.y["S1_STI"] + args["mu"] * args["N0s"][0]
    cm.dy["Ia1_STI"] = cm.dy["Ia1_STI"] - args["mu"] * cm.y["Ia1_STI"]
    cm.dy["Is1_STI"] = cm.dy["Is1_STI"] - args["mu"] * cm.y["Is1_STI"] 
    cm.dy["T1_STI"] = cm.dy["T1_STI"] - args["mu"] * cm.y["T1_STI"]

    cm.flow("S2_STI", "Ia2_STI", infect_ia2(y, args))  # Susceptible to asymptomatic
    cm.flow("S2_STI", "Is2_STI", infect_is2(y, args))  # Susceptible to symptomatic
    cm.flow("Ia2_STI", "S2_STI", args["gamma_STI"])  # Asymptomatic to susceptible (recovery)
    cm.flow("Ia2_STI", "T2_STI", lambda_a(args))  # Asymptomatic to tested and treatment
    cm.flow("Is2_STI", "T2_STI", args["lambda_s"])  # Symptomatic to tested and treatment
    cm.flow("T2_STI", "S2_STI", args["gammaT_STI"])  # Treatment to susceptible (immunity loss)

    # Vital dynamics (New addition/removal to/from risk group)
    cm.dy["S2_STI"] = cm.dy["S2_STI"] - args["mu"] * cm.y["S2_STI"] + args["mu"] * args["N0s"][1]
    cm.dy["Ia2_STI"] = cm.dy["Ia2_STI"] - args["mu"] * cm.y["Ia2_STI"]
    cm.dy["Is2_STI"] = cm.dy["Is2_STI"] - args["mu"] * cm.y["Is2_STI"]
    cm.dy["T2_STI"] = cm.dy["T2_STI"] - args["mu"] * cm.y["T2_STI"]

    cm.flow("S3_STI", "Ia3_STI", infect_ia3(y, args))  # Susceptible to asymptomatic
    cm.flow("S3_STI", "Is3_STI", infect_is3(y, args))  # Susceptible to symptomatic
    cm.flow("Ia3_STI", "S3_STI", args["gamma_STI"])  # Asymptomatic to susceptible (recovery)
    cm.flow("Ia3_STI", "T3_STI", lambda_a(args))  # Asymptomatic to tested and treatment
    cm.flow("Is3_STI", "T3_STI", args["lambda_s"])  # Symptomatic to tested and treatment
    cm.flow("T3_STI", "S3_STI", args["gammaT_STI"])  # Treatment to susceptible (immunity loss)

    # Vital dynamics (New addition/removal to/from risk group)
    cm.dy["S3_STI"] = cm.dy["S3_STI"] - args["mu"] * cm.y["S3_STI"] + args["mu"] * args["N0s"][2]
    cm.dy["Ia3_STI"] = cm.dy["Ia3_STI"] - args["mu"] * cm.y["Ia3_STI"]
    cm.dy["Is3_STI"] = cm.dy["Is3_STI"] - args["mu"] * cm.y["Is3_STI"]
    cm.dy["T3_STI"] = cm.dy["T3_STI"] - args["mu"] * cm.y["T3_STI"]

    cm.flow("S4_STI", "Ia4_STI", infect_ia4(y, args))  # Susceptible to asymptomatic
    cm.flow("S4_STI", "Is4_STI", infect_is4(y, args))  # Susceptible to symptomatic
    cm.flow("Ia4_STI", "S4_STI", args["gamma_STI"])  # Asymptomatic to susceptible (recovery)
    cm.flow("Ia4_STI", "T4_STI", lambda_a(args))  # Asymptomatic to tested and treatment
    cm.flow("Is4_STI", "T4_STI", args["lambda_s"])  # Symptomatic to tested and treatment
    cm.flow("T4_STI", "S4_STI", args["gammaT_STI"])  # Treatment to susceptible (immunity loss)

    # Vital dynamics (New addition/removal to/from risk group)
    cm.dy["S4_STI"] = cm.dy["S4_STI"] - args["mu"] * cm.y["S4_STI"] + args["mu"] * args["N0s"][3]
    cm.dy["Ia4_STI"] = cm.dy["Ia4_STI"] - args["mu"] * cm.y["Ia4_STI"]
    cm.dy["Is4_STI"] = cm.dy["Is4_STI"] - args["mu"] * cm.y["Is4_STI"]
    cm.dy["T4_STI"] = cm.dy["T4_STI"] - args["mu"] * cm.y["T4_STI"]

    # Return the differential changes
    return cm.dy

# Function to setup the model and return the integrator
def setup_model(args, y0):
    """
    Set up the model for simulation.

    Args:
        args: Additional arguments for setting up the model.
        y0: Initial conditions for the model.

    Returns:
        integrator: A function that can be used to solve the ODEs defined in the 'model' function.
    """

    # Define the time span for the simulation
    ts = np.linspace(0, 3600 * 10, 3600)

    # Create an ODE integrator object using the icomo library
    integrator_object = icomo.ODEIntegrator(
        ts_out=ts,  # Output time points
        t_0=min(ts),  # Initial time point
        ts_solver=ts,  # Time points for the solver to use
    )

    # Get the integration function for the model
    integrator = integrator_object.get_func(
        model
    )  # Returns a function that can be used to solve the ODEs defined in the 'model' function

    logger.info("Model setup complete and ready for simulation")

    return integrator


# If this script is run directly, setup the model
# if __name__ == "__main__":
#     integrator, y0, args = setup_model()
